<a href="https://colab.research.google.com/github/lai86423/1018/blob/master/PT_4_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python

# Any copyright is dedicated to the Public Domain.
# https://creativecommons.org/publicdomain/zero/1.0/

# Written by Francois Fleuret <francois@fleuret.org>

import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

train_input, train_target, test_input, test_target = \
    prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)

######################################################################

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x

* Using MNIST


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



...

Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [ ]:
model, criterion = Net(), nn.MSELoss()
eta, mini_batch_size = 1e-1, 100
nb_epochs = 100

for e in range(nb_epochs):
    acc_loss = 0
    # We do this with mini-batches
    for b in range(0, train_input.size(0), mini_batch_size):
        output = model(train_input.narrow(0, b, mini_batch_size))
        loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
        acc_loss = acc_loss + loss.item()

        model.zero_grad()
        loss.backward()
        with torch.no_grad():
            for p in model.parameters():
                p -= eta * p.grad

    print(e, acc_loss)


* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples
0 0.9114229157567024
1 0.7867081984877586
2 0.7167952582240105
3 0.6580886617302895
4 0.6101822257041931
5 0.577421773225069
6 0.5493221431970596
7 0.5108993351459503
8 0.4806525185704231
9 0.45650114491581917
10 0.4710356555879116
11 0.4329895041882992
12 0.39128659293055534
13 0.38951123133301735
14 0.37185826897621155
15 0.35115284845232964
16 0.3386825118213892
17 0.3295481204986572
18 0.33267495408654213
19 0.32770725153386593
20 0.3062488231807947
21 0.2994359489530325
22 0.29126142524182796
23 0.27888680063188076
24 0.26889482513070107
25 0.2668888531625271
26 0.2639841139316559
27 0.2489670142531395
28 0.24328235536813736
29 0.2511893119663
30 0.23813919723033905
31 0.22518973983824253
32 0.22360952012240887
33 0.22972544841468334
34 0.22093810513615608
35 0.20846786350011826
36 0.20336448773741722
37 0.20771942846477032
38 0.20775813050568104
39 0.19398969132453203
40 0

In [ ]:
#1
def train_model(model, train_input, train_target, mini_batch_size):
    # We do this with mini-batches
    acc_loss = 0
    for b in range(0, train_input.size(0), mini_batch_size):
        output = model(train_input.narrow(0, b, mini_batch_size)) #表示取出train_input中第0维上索引从b开始到index+mini_batch_size-1的所有元素
        loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
        acc_loss = acc_loss + loss.item()
        model.zero_grad()
        loss.backward()
        with torch.no_grad():
            for p in model.parameters():
                p -= eta * p.grad

    return model, acc_loss

In [ ]:
def compute_nb_errors(model, input, target, mini_batch_size):
  with torch.no_grad(): 
    output = model(input)
    error = 0
    for i in range(input.size(0)):
      if torch.argmax(output[i])!= torch.argmax(target[i]):
          error += 1
    Error = error/input.size(0)
    print("error=", Error)


In [ ]:
model = Net()
eta, mini_batch_size = 1e-1, 100
nb_epochs = 100
criterion = nn.MSELoss()
for e in range(nb_epochs):
    model, acc_loss = train_model(model, train_input, train_target, mini_batch_size)
    compute_nb_errors(model, test_input, test_target, mini_batch_size)
    #print(e, acc_loss)


error= 0.716
error= 0.488
error= 0.415
error= 0.375
error= 0.342
error= 0.308
error= 0.26
error= 0.251
error= 0.234
error= 0.216
error= 0.215
error= 0.213
error= 0.206
error= 0.234
error= 0.233
error= 0.249
error= 0.193
error= 0.187
error= 0.187
error= 0.187
error= 0.188
error= 0.167
error= 0.174
error= 0.152
error= 0.144
error= 0.147
error= 0.134
error= 0.138
error= 0.148
error= 0.132
error= 0.144
error= 0.117
error= 0.115
error= 0.116
error= 0.111
error= 0.107
error= 0.136
error= 0.107
error= 0.135
error= 0.108
error= 0.107
error= 0.11
error= 0.108
error= 0.105
error= 0.115
error= 0.103
error= 0.093
error= 0.098
error= 0.088
error= 0.087
error= 0.087
error= 0.095
error= 0.099
error= 0.104
error= 0.093
error= 0.084
error= 0.082
error= 0.075
error= 0.076
error= 0.076
error= 0.082
error= 0.092
error= 0.097
error= 0.094
error= 0.077
error= 0.075
error= 0.068
error= 0.071
error= 0.07
error= 0.071
error= 0.072
error= 0.071
error= 0.08
error= 0.084
error= 0.083
error= 0.075
error= 0.07
erro